In [1]:
!trtexec

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec
=== Model Options ===
  --uff=<file>                UFF model
  --onnx=<file>               ONNX model
  --model=<file>              Caffe model (default = no model, random weights used)
  --deploy=<file>             Caffe prototxt file
  --output=<name>[,<name>]*   Output names (it can be specified multiple times); at least one output is required for UFF and Caffe
  --uffInput=<name>,X,Y,Z     Input blob name and its dimensions (X,Y,Z=C,H,W), it can be specified multiple times; at least one is required for UFF models
  --uffNHWC                   Set if inputs are in the NHWC layout instead of NCHW (use X,Y,Z=H,W,C order in --uffInput)

=== Build Options ===
  --maxBatch                  Set max batch size and build an implicit batch engine (default = same size as --batch)
                              This option should not be used when the input model is ONNX or when dynamic shapes are provided.
  --minShapes=spec            B

In [2]:
import os

In [3]:
ONNX_MODEL_DIR = './Flask/Models/'

FLASK_MODEL_DIR = './Flask/Models/'

TRITON_MODEL_DIR = './Triton/Models/'
TRITON_CONFIG_FILE = 'config.pbtxt'
TRITON_MODEL_FILE = 'model.plan'

In [4]:
def save_tensorrt_model(onnx_model, trt_model, precision, config):
    trt_model = trt_model + '_' + precision
    onnx_model_path = os.path.join(ONNX_MODEL_DIR, onnx_model + '.onnx')
    flask_model_path = os.path.join(FLASK_MODEL_DIR, trt_model + '.plan')

    !trtexec \
        --onnx=$onnx_model_path \
        --explicitBatch \
        --$precision \
        --saveEngine=$flask_model_path

    triton_config_path = os.path.join(TRITON_MODEL_DIR, trt_model, TRITON_CONFIG_FILE)
    os.makedirs(os.path.dirname(triton_config_path), exist_ok=True)
    with open(triton_config_path, 'w') as f:
        f.write(config.strip())

    triton_model_path = os.path.join(TRITON_MODEL_DIR, trt_model, '1', TRITON_MODEL_FILE)
    os.makedirs(os.path.dirname(triton_model_path), exist_ok=True)
    !cp $flask_model_path $triton_model_path
        
    print(flask_model_path)
    print(triton_config_path)
    print(triton_model_path)
    return

In [5]:
onnx_model = 'resnet34-prune20-onnx'
trt_model = 'resnet34-prune20-trt'
precision = 'best'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec --onnx=./Flask/Models/resnet34-prune20-onnx.onnx --explicitBatch --best --saveEngine=./Flask/Models/resnet34-prune20-trt_best.plan
[10/20/2022-09:22:27] [W] --explicitBatch flag has been deprecated and has no effect!
[10/20/2022-09:22:27] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[10/20/2022-09:22:27] [I] === Model Options ===
[10/20/2022-09:22:27] [I] Format: ONNX
[10/20/2022-09:22:27] [I] Model: ./Flask/Models/resnet34-prune20-onnx.onnx
[10/20/2022-09:22:27] [I] Output:
[10/20/2022-09:22:27] [I] === Build Options ===
[10/20/2022-09:22:27] [I] Max batch: explicit batch
[10/20/2022-09:22:27] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[10/20/2022-09:22:27] [I] minTiming: 1
[10/20/2022-09:22:27] [I] avgTiming: 8
[10/20/2022-09:22:27] [I] Precision: FP32+FP16+INT8
[10/20/2022-09:22:27] [

[10/20/2022-09:22:42] [W] [TRT] Weights [name=Conv_20 + Add_22 + Relu_23.weight] had the following issues when converted to FP16:
[10/20/2022-09:22:42] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:22:42] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:22:42] [W] [TRT] Weights [name=Conv_20 + Add_22 + Relu_23.weight] had the following issues when converted to FP16:
[10/20/2022-09:22:42] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:22:42] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:22:48] [W] [TRT] Weights [name=Conv_41 + Add_43 + Relu_44.weight] had the following issues when converted to FP16:
[10/20/2022-09:22:48] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:22:48] [W] [TRT] If this is not the desired behavior, pleas

[10/20/2022-09:22:57] [W] [TRT] Weights [name=Conv_72 + Add_74 + Relu_75.weight] had the following issues when converted to FP16:
[10/20/2022-09:22:57] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:22:57] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:22:57] [W] [TRT] Weights [name=Conv_72 + Add_74 + Relu_75.weight] had the following issues when converted to FP16:
[10/20/2022-09:22:57] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:22:57] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:23:03] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/20/2022-09:23:03] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:03] [W] [TRT] If this is not the desired behavior, please modify the weight

[10/20/2022-09:23:03] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/20/2022-09:23:03] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:03] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:23:03] [W] [TRT] Weights [name=Gemm_88.bias] had the following issues when converted to FP16:
[10/20/2022-09:23:03] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:03] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:23:03] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/20/2022-09:23:03] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:03] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to redu

./Flask/Models/resnet34-prune20-trt_best.plan
./Triton/Models/resnet34-prune20-trt_best/config.pbtxt
./Triton/Models/resnet34-prune20-trt_best/1/model.plan


In [6]:
onnx_model = 'resnet34-prune50-onnx'
trt_model = 'resnet34-prune50-trt'
precision = 'best'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec --onnx=./Flask/Models/resnet34-prune50-onnx.onnx --explicitBatch --best --saveEngine=./Flask/Models/resnet34-prune50-trt_best.plan
[10/20/2022-09:23:08] [W] --explicitBatch flag has been deprecated and has no effect!
[10/20/2022-09:23:08] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[10/20/2022-09:23:08] [I] === Model Options ===
[10/20/2022-09:23:08] [I] Format: ONNX
[10/20/2022-09:23:08] [I] Model: ./Flask/Models/resnet34-prune50-onnx.onnx
[10/20/2022-09:23:08] [I] Output:
[10/20/2022-09:23:08] [I] === Build Options ===
[10/20/2022-09:23:08] [I] Max batch: explicit batch
[10/20/2022-09:23:08] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[10/20/2022-09:23:08] [I] minTiming: 1
[10/20/2022-09:23:08] [I] avgTiming: 8
[10/20/2022-09:23:08] [I] Precision: FP32+FP16+INT8
[10/20/2022-09:23:08] [

[10/20/2022-09:23:22] [W] [TRT] Weights [name=Conv_20 + Add_22 + Relu_23.weight] had the following issues when converted to FP16:
[10/20/2022-09:23:22] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:22] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:23:22] [W] [TRT] Weights [name=Conv_20 + Add_22 + Relu_23.weight] had the following issues when converted to FP16:
[10/20/2022-09:23:22] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:22] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:23:28] [W] [TRT] Weights [name=Conv_41 + Add_43 + Relu_44.weight] had the following issues when converted to FP16:
[10/20/2022-09:23:28] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:28] [W] [TRT] If this is not the desired behavior, pleas

[10/20/2022-09:23:37] [W] [TRT] Weights [name=Conv_72 + Add_74 + Relu_75.weight] had the following issues when converted to FP16:
[10/20/2022-09:23:37] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:37] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:23:37] [W] [TRT] Weights [name=Conv_72 + Add_74 + Relu_75.weight] had the following issues when converted to FP16:
[10/20/2022-09:23:37] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:37] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:23:43] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/20/2022-09:23:43] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:43] [W] [TRT] If this is not the desired behavior, please modify the weight

[10/20/2022-09:23:43] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/20/2022-09:23:43] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:43] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:23:43] [W] [TRT] Weights [name=Gemm_88.bias] had the following issues when converted to FP16:
[10/20/2022-09:23:43] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:43] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:23:43] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/20/2022-09:23:43] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:23:43] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to redu

./Flask/Models/resnet34-prune50-trt_best.plan
./Triton/Models/resnet34-prune50-trt_best/config.pbtxt
./Triton/Models/resnet34-prune50-trt_best/1/model.plan


In [7]:
onnx_model = 'resnet34-prune70-onnx'
trt_model = 'resnet34-prune70-trt'
precision = 'best'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec --onnx=./Flask/Models/resnet34-prune70-onnx.onnx --explicitBatch --best --saveEngine=./Flask/Models/resnet34-prune70-trt_best.plan
[10/20/2022-09:23:48] [W] --explicitBatch flag has been deprecated and has no effect!
[10/20/2022-09:23:48] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[10/20/2022-09:23:48] [I] === Model Options ===
[10/20/2022-09:23:48] [I] Format: ONNX
[10/20/2022-09:23:48] [I] Model: ./Flask/Models/resnet34-prune70-onnx.onnx
[10/20/2022-09:23:48] [I] Output:
[10/20/2022-09:23:48] [I] === Build Options ===
[10/20/2022-09:23:48] [I] Max batch: explicit batch
[10/20/2022-09:23:48] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[10/20/2022-09:23:48] [I] minTiming: 1
[10/20/2022-09:23:48] [I] avgTiming: 8
[10/20/2022-09:23:48] [I] Precision: FP32+FP16+INT8
[10/20/2022-09:23:48] [

[10/20/2022-09:24:02] [W] [TRT] Weights [name=Conv_20 + Add_22 + Relu_23.weight] had the following issues when converted to FP16:
[10/20/2022-09:24:02] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:24:02] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:24:02] [W] [TRT] Weights [name=Conv_20 + Add_22 + Relu_23.weight] had the following issues when converted to FP16:
[10/20/2022-09:24:02] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:24:02] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:24:08] [W] [TRT] Weights [name=Conv_41 + Add_43 + Relu_44.weight] had the following issues when converted to FP16:
[10/20/2022-09:24:08] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:24:08] [W] [TRT] If this is not the desired behavior, pleas

[10/20/2022-09:24:17] [W] [TRT] Weights [name=Conv_72 + Add_74 + Relu_75.weight] had the following issues when converted to FP16:
[10/20/2022-09:24:17] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:24:17] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:24:17] [W] [TRT] Weights [name=Conv_72 + Add_74 + Relu_75.weight] had the following issues when converted to FP16:
[10/20/2022-09:24:17] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:24:17] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:24:23] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/20/2022-09:24:23] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:24:23] [W] [TRT] If this is not the desired behavior, please modify the weight

[10/20/2022-09:24:23] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/20/2022-09:24:23] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:24:23] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:24:23] [W] [TRT] Weights [name=Gemm_88.bias] had the following issues when converted to FP16:
[10/20/2022-09:24:23] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:24:23] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/20/2022-09:24:23] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/20/2022-09:24:23] [W] [TRT]  - Subnormal FP16 values detected. 
[10/20/2022-09:24:23] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to redu

./Flask/Models/resnet34-prune70-trt_best.plan
./Triton/Models/resnet34-prune70-trt_best/config.pbtxt
./Triton/Models/resnet34-prune70-trt_best/1/model.plan
